In [ ]:
import gensim
import networkx as nx
from networkx.algorithms import community
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

m = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

words = ['РєРёСЃР»С‹Р№_A', 'РіРѕСЂСЊРєРёР№_A']
words2 = []
# РјР°СЃСЃРёРІ, РІ РєРѕС‚РѕСЂС‹Р№ Р±СѓРґСѓС‚ Р·Р°РїРёСЃР°РЅС‹ СЃР»РѕРІР° РІС‚РѕСЂРѕРіРѕ РїРѕСЂСЏРґРєР°

G = nx.Graph()
G.add_node('РєРёСЃР»С‹Р№')
G.add_node('РіРѕСЂСЊРєРёР№')


def func(word):
    res = []
    # РІС‹РґР°РµРј 10 Р±Р»РёР¶Р°Р№С€РёС… СЃРѕСЃРµРґРµР№ СЃР»РѕРІР°:
    for i in model.most_similar(positive=[word], topn=50):
        # СЃР»РѕРІРѕ + РєРѕСЌС„С„РёС†РёРµРЅС‚ РєРѕСЃРёРЅСѓСЃРЅРѕР№ Р±Р»РёР·РѕСЃС‚Рё
        # print(i[0], i[1])
        if i[0].endswith('_A'):
            # РїСЂРѕРІРµСЂСЏРµРј С‡С‚Рѕ СЌС‚Рѕ РїСЂРёР»Р°РіР°С‚РµР»СЊРЅРѕРµ
            if i[1] >= 0.5:
                # РїСЂРѕРІРµСЂСЏРµРј С‡С‚Рѕ РєРѕСЌС„С„РёС†РёРµРЅС‚ Р±РѕР»СЊС€Рµ 0.5
                if not i[0][:-2] in G.nodes():
                    # РґРѕР±Р°РІР»СЏРµРј СѓР·РµР» РµСЃР»Рё РѕРЅ РµС‰Рµ РЅРµ СЃСѓС‰РµСЃС‚РІРѕРІР°Р»
                    G.add_node(i[0][:-2])
                    res.append(i[0])
                G.add_edge(word[:-2], (i[0][:-2]))
    if i[1] >= 0.5:
        # РїСЂРѕРІРµСЂРєР° РµСЃР»Рё РІС‹РІРµРґРµРЅС‹ РЅРµ РІСЃРµ СЃР»РѕРІР° СЃ Р±РѕР»СЊС€РёРј РєРѕСЌС„С„.
        print('go deeper')
    return res

for w in words:
    # РЅР°С…РѕРґРёРј СЃРѕСЃРµРґРµР№ РґР»СЏ СЃР»РѕРІ РїРµСЂРІРѕРіРѕ РїРѕСЂСЏРґРєР°
    interme = func(w)
    for i in interme:
        # РґРѕР±Р°РІР»СЏРµРј РІСЃРµ СЃР»РѕРІР° РІС‚РѕСЂРѕРіРѕ РїРѕСЂСЏРґРєР° РІ РјР°СЃСЃРёРІ
        if i not in words2:
            words2.append(i)

for w in words2:
    # РЅР°С…РѕРґРёРј СЃРѕСЃРµРґРµР№ РґР»СЏ СЃР»РѕРІ РІС‚РѕСЂРѕРіРѕ РїРѕСЂСЏРґРєР°
    func(w)

deg = nx.degree_centrality(G)
central_deg = sorted(deg, key=deg.get, reverse=True)[0]
print('Central node based on degree centrality - ' + central_deg)

clo = nx.closeness_centrality(G)
central_clo = sorted(clo, key=clo.get, reverse=True)[0]
print('Central node based on closeness centrality - ' + central_clo)

bet = nx.betweenness_centrality(G)
central_bet = sorted(bet, key=bet.get, reverse=True)[0]
print('Central node based on betweenness centrality - ' + central_bet)

eig = nx.eigenvector_centrality(G)
central_eig = sorted(eig, key=eig.get, reverse=True)[0]
print('Central node based on eigencentrality - ' + central_eig)

print('\n')
print('Density - ' + str(nx.density(G)))
print('Diameter - ' + str(nx.diameter(G)))
print('Radius - ' + str(nx.radius(G)))
print('Clustering coefficient - ' + str(nx.average_clustering(G)))
print('Assortativity coefficient - ' +
      str(nx.degree_pearson_correlation_coefficient(G)))
print('\n')

# СЂР°Р·Р±РёРІР°РµРј РЅР° СЃРѕРѕР±С‰РµСЃС‚РІР°
com = community.greedy_modularity_communities(G)
print('COMMUNITIES')
for acom in com:
    print(acom)
# СЃРѕРѕР±С‰РµСЃС‚РІР° СЂР°Р·РґРµР»РµРЅС‹ РїРѕ Р±Р»РёР·РѕСЃС‚Рё РїРѕ Р·РЅР°С‡РµРЅРёСЋ СЃРѕ СЃР»РѕРІР°РјРё 2 РїРѕСЂСЏРґРєР°
# 1-Рµ РїРѕ СЃР»РѕРІСѓ РєРёСЃР»РѕРІР°С‚С‹Р№
# 2-Рµ РїРѕ СЃР»РѕРІСѓ СЃР»Р°РґРєРёР№
# 3-Рµ РїРѕ СЃР»РѕРІСѓ РіРѕСЂРµСЃС‚РЅС‹Р№

# РІРёР·СѓР°Р»РёР·Р°С†РёСЏ
pos = nx.spring_layout(G)
nx.draw_networkx_edges(G, pos, edge_color='red')
nx.draw_networkx_nodes(G, pos, node_color='black', node_size=10)
nx.draw_networkx_labels(G, pos, font_color='black',
                        font_size=10, font_family='Arial')
plt.axis('off')
plt.show()
